<a href="https://colab.research.google.com/github/AdrianoHKim/Soulcode/blob/main/notebooks/notebook_docs_yaml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Equipe 08**

**Integrantes**: Wesley, Marcos Paulo, Adriano Kim, Paulo Vitorino, Renato e José Aurelio

Professores: Douglas Ribeiro, Franciane Rodrigues e Jonathas Carneiro

## Preparação de Ambiente

In [ ]:
# Instalaçao Bibliotecas
!pip install gcsfs -q

In [ ]:
# Importando Bibliotecas
import os
import pandas as pd

from google.cloud import storage
from google.colab import data_table
from google.colab import drive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [ ]:
# Ignorando alguns alertas desnecessários
import warnings
warnings.filterwarnings("ignore")

### Configuração PySpark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -N -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql.functions import regexp_replace
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Para deixar a visualição das tabelas mais amigável

import pyspark.sql.functions as F
from pyspark.sql.types import *

spark

### Compartilhamento Chave GDrive

In [ ]:
# Cria compartilhamento com Google Drive
drive.mount('/content/drive', force_remount=True)

# Arquivo a ser acessado na pasta compartilhada

target = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Caminho completo da pasta compartilhada
folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

# Acesso ao arquivo no colab
serviceAccount = os.path.join(folder, target)

Mounted at /content/drive


### Configuração da bucket

In [ ]:
# Nome da bucket
bucket_name = 'projeto-final-ad2-e8'

# Delimitação de pasta
prefix_name = 'dados/tratados/'

In [ ]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# Conexão com a bucket do Google Cloud
client = storage.Client()
bucket = client.get_bucket(bucket_name)

### Conjuntos de dados

In [ ]:
# Habilita formatação tabular
data_table.enable_dataframe_formatter()

In [ ]:
# Listando conjuntos de dados na bucket
blobList = [x.name for x in bucket.list_blobs(prefix = prefix_name)]

# dataframe lista blobs
blobDf = pd.DataFrame(blobList)

blobDf

,0
0,dados/tratados/
1,dados/tratados/dataset_tratados_xlsx/
2,dados/tratados/dataset_tratados_xlsx/2_1_cap_i...
3,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
4,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
5,dados/tratados/dataset_tratados_xlsx/2_3_gerac...
6,dados/tratados/fator_capacidade_mwh_2015_2023.csv
7,dados/tratados/iea_ponto_recarga_tratado.csv
8,dados/tratados/iea_veiculos_tratado.csv
9,dados/tratados/tratado_geracao_distribuida.csv


In [ ]:
# Desabilita formatação tabular
data_table.disable_dataframe_formatter()

In [ ]:
# Renomeando coluna
blobDf.columns = ['path']

# Lista com resultados
blobPath = blobDf['path'].tolist()

#### Seleção de dados

In [ ]:
# Seleção por index conforme caminho da tabela acima
index = int(input('Infome o número do índice correspondente ao arquivo, conforme tabela acima:' ))

# Seleção do gsutil do arquivo
path = f'gs://{bucket_name}/{blobPath[index]}'

archive= blobPath[index][len(blobPath[0]):]

print(f'>>> arquivo : {archive}\n>>> gsutil: {path}')

Infome o número do índice correspondente ao arquivo, conforme tabela acima:6
>>> arquivo : fator_capacidade_mwh_2015_2023.csv
>>> gsutil: gs://projeto-final-ad2-e8/dados/tratados/fator_capacidade_mwh_2015_2023.csv


In [ ]:
# Configuração do blob
blob = bucket.blob(blobPath[index])

# Download do arquivo
blob.download_to_filename('/content/' + archive)

In [ ]:
downloadFile = '/content/fator_capacidade_mwh_2015_2023.csv'

### Funções

#### Extração de Schema

In [ ]:
# função de definição do Schema
def docSchema(nome_dataset, dataframe, formato):
  '''Após a criação do schema e preenchimento das informações em um editor
  de texto, passar o conteúdo para validar no site: https://www.yamllint.com/'''
# modelo de impressão dos dados
  print(f"""
nome: {nome_dataset}
descricao:
origem:
tratamento: squadOito
armazenamento: GCS
acesso: privado
formato: {formato}
schema:""")
# laço de repetição para todas as colunas do dataframe
  for columns in dataframe.dtypes:
    print(f""" - coluna: {columns[0]}
   descricao:
   tipo: {columns[1]}""")

## Documentação

### Leitura do dataframe

In [ ]:
df = spark.read.csv(downloadFile, header=True, inferSchema=True)

### Chamando função

In [ ]:
docSchema('fator_capacidade', df, 'csv')


nome: fator_capacidade
descricao:
origem:
tratamento: squadOito
armazenamento: GCS
acesso: privado
formato: csv
schema:
 - coluna: id_subsistema
   descricao:
   tipo: string
 - coluna: nom_subsistema
   descricao:
   tipo: string
 - coluna: id_estado
   descricao:
   tipo: string
 - coluna: nom_estado
   descricao:
   tipo: string
 - coluna: nom_pontoconexao
   descricao:
   tipo: string
 - coluna: nom_localizacao
   descricao:
   tipo: string
 - coluna: val_latitudesecoletora
   descricao:
   tipo: double
 - coluna: val_longitudesecoletora
   descricao:
   tipo: double
 - coluna: val_latitudepontoconexao
   descricao:
   tipo: double
 - coluna: val_longitudepontoconexao
   descricao:
   tipo: double
 - coluna: nom_modalidadeoperacao
   descricao:
   tipo: string
 - coluna: nom_tipousina
   descricao:
   tipo: string
 - coluna: nom_usina_conjunto
   descricao:
   tipo: string
 - coluna: id_ons
   descricao:
   tipo: string
 - coluna: ceg
   descricao:
   tipo: string
 - coluna: data


# Encerramento

In [ ]:
# encerrando o SparkSession
#spark.stop()